In [128]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import glob
import os
import numpy as np
import datetime

In [2]:
os.chdir(r'C:\NICE Documents\Bank of Indonesia\Raw Data')

In [3]:
data = pd.concat(map(pd.read_csv, glob.glob('*.csv')))

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3319: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
data.shape

(12949324, 77)

In [7]:
del data

In [410]:
data_temp = data[data.Type=='Trade']

NameError: name 'data' is not defined

In [8]:
data_temp = data_temp[['#RIC','Date-Time','Price','Volume','Qualifiers']]

In [414]:
data_temp.Qualifiers = data_temp.Qualifiers.map(lambda x: x.split(';')[1][1])

IndexError: list index out of range

In [468]:
def create_date_featues(df):
    df['Date-Time'] = pd.to_datetime(df['Date-Time'],format='%Y-%m-%d')
    df['Year'] = pd.to_datetime(df['Date-Time']).dt.year
    df['Month'] = pd.to_datetime(df['Date-Time']).dt.month
    df['Day'] = pd.to_datetime(df['Date-Time']).dt.day
    df['Hour'] = pd.to_datetime(df['Date-Time']).dt.hour
    return df

In [469]:
df = create_date_featues(data_temp)

In [470]:
df.columns

Index(['#RIC', 'Date-Time', 'Price', 'Volume', 'Qualifiers', 'Turnover',
       'Year', 'Month', 'Day', 'Hour', 'Minutes', 'Second', 'row_num',
       'DateTime', 'Date-Hour'],
      dtype='object')

In [516]:
df_buy = df[df.Qualifiers=='b']
data_Grp_buy = df_buy.groupby(['#RIC','Year','Month','Day','Hour'])

In [517]:
df_buy_features = data_Grp_buy.Price.sum().reset_index().rename(columns={'Price': 'buy_price_sum'})
df_buy_features['buy_vol_sum'] = data_Grp_buy.Volume.sum().values

In [518]:
df_buy.columns

Index(['#RIC', 'Date-Time', 'Price', 'Volume', 'Qualifiers', 'Turnover',
       'Year', 'Month', 'Day', 'Hour', 'Minutes', 'Second', 'row_num',
       'DateTime', 'Date-Hour'],
      dtype='object')

In [519]:
df_sale = df[df.Qualifiers=='s']
data_Grp_sale = df_sale.groupby(['#RIC','Year','Month','Day','Hour'])

In [520]:
df_sale_features = data_Grp_sale.Price.sum().reset_index().rename(columns={'Price': 'sale_price_sum'})
df_sale_features['sale_vol_sum'] = data_Grp_sale.Volume.sum().values

In [503]:
df_sale.columns

Index(['#RIC', 'Year', 'Month', 'Day', 'Hour', 'sale_price_sum',
       'sale_vol_sum'],
      dtype='object')

In [457]:
agg_data.shape

(8324, 18)

In [541]:
def create_features(df, tag):
    data_Grp = df.groupby(['#RIC','Year','Month','Day','Hour'])
    agg_data = data_Grp.Price.min().reset_index()
    agg_data['max_price'] = data_Grp.Price.max().values
    agg_data['total_price'] = data_Grp.Price.sum().values
    agg_data['mean_price'] = data_Grp.Price.mean().values
    agg_data['price_%change']=agg_data.mean_price.pct_change().fillna(0)
    agg_data['max_volume'] = data_Grp.Volume.max().values
    agg_data['total_volume'] = data_Grp.Volume.sum().values
    agg_data['min_volume'] = data_Grp.Volume.min().values
    agg_data['mean_volume'] = data_Grp.Volume.mean().values
    agg_data['mean_vol_%change']=agg_data.mean_volume.pct_change().fillna(0)
    agg_data.rename(columns={'Price':'min_price'},inplace=True)
    agg_data['vwap_price'] = data_Grp.Turnover.sum().values/agg_data['mean_volume']
    
    agg_data_f = pd.DataFrame([])
    for r in list(agg_data['#RIC'].unique()):
        agg_data_t = agg_data[agg_data['#RIC']==r]
        agg_data_t['rolling_mean_price']= agg_data_t.mean_price.rolling(8, min_periods=1).mean()
        agg_data_t['rolling_mean_vol']= agg_data_t.mean_volume.rolling(8, min_periods=1).mean()
        agg_data_t['rolling_mean_vwap']= agg_data_t.vwap_price.rolling(8, min_periods=1).mean()
        agg_data_t['mean_price_t_1']= agg_data_t.mean_price.shift(1).fillna(agg_data_t.mean_price)
        agg_data_t['mean_volume_t_1']= agg_data_t.mean_volume.shift(1).fillna(agg_data_t.mean_volume)
        agg_data_f= pd.concat([agg_data_f,agg_data_t],axis=0, ignore_index=True)
        
    agg_data_f['price_%change_trend8hr'] = (agg_data_f.mean_price - agg_data_f.mean_price_t_1)*100/agg_data_f.rolling_mean_price
    agg_data_f['mean_vol_%change_trend8hr'] = (agg_data_f.mean_volume - agg_data_f.mean_volume_t_1)*100/agg_data_f.rolling_mean_vol
    agg_data_f['vwap_price_trend8hr'] = agg_data_f.vwap_price/agg_data_f.rolling_mean_vwap
    
    agg_data_final = agg_data_f[['#RIC','Year','Month', 'Day', 'Hour','max_price','min_price',
       'mean_price', 'price_%change', 'price_%change_trend8hr', 'max_volume', 
       'min_volume', 'mean_volume', 'mean_vol_%change','mean_vol_%change_trend8hr', 'vwap_price', 'vwap_price_trend8hr']]
    
    agg_data_final.rename(columns = {'max_price': tag + '_' + 'max_price',
                                     'min_price': tag + '_' + 'min_price',
                                     'mean_price': tag + '_' + 'mean_price',
                                     'price_%change': tag + '_' + 'price_%change',
                                     'price_%change_trend8hr': tag + '_' + 'price_%change_trend8hr',
                                     'max_volume': tag + '_' + 'max_volume',
                                     'min_volume': tag + '_' + 'min_volume',
                                     'mean_volume': tag + '_' + 'mean_volume',
                                     'mean_vol_%change': tag + '_' + 'mean_vol_%change',
                                     'mean_vol_%change_trend8hr': tag + '_' + 'mean_vol_%change_trend8hr',
                                     'vwap_price': tag + '_' + 'vwap_price',
                                     'vwap_price_trend8hr': tag + '_' + 'vwap_price_trend8hr',
                                    }, inplace=True)
    return agg_data_final

In [542]:
agg_data_buy = create_features(df_buy, "buy")
agg_data_sale = create_features(df_sale, "sale")
agg_data_all = create_features(df, "total")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [543]:
agg_data_all = agg_data_all.merge(df_buy_features, on= ['#RIC', 'Year','Month','Day', 'Hour'], how='left')

In [544]:
agg_data_all = agg_data_all.merge(df_sale_features, on= ['#RIC','Year','Month', 'Day', 'Hour'], how='left')

In [545]:
agg_data_all['buy-sell-price_%change8hr'] = (agg_data_all.buy_price_sum - agg_data_all.sale_price_sum)*100/(agg_data_all.buy_price_sum + agg_data_all.sale_price_sum)

In [546]:
agg_data_all['buy-sell-volume_%change'] = (agg_data_all.buy_vol_sum - agg_data_all.sale_vol_sum)*100/(agg_data_all.buy_vol_sum + agg_data_all.sale_vol_sum)

In [547]:
agg_data_all = agg_data_all.drop(['buy_price_sum', 'buy_vol_sum', 'sale_price_sum', 'sale_vol_sum'], axis=1)

In [548]:
agg_data_final = agg_data_all.merge(agg_data_buy, on= ['#RIC', 'Year','Month','Day', 'Hour'], how='left')
agg_data_final = agg_data_final.merge(agg_data_sale, on= ['#RIC', 'Year','Month','Day', 'Hour'], how='left')

In [549]:
agg_data_final[agg_data_final.isnull()].shape

(8324, 43)

In [564]:
agg_data_final = agg_data_final.dropna()

In [565]:
agg_data_final

,#RIC,Year,Month,Day,Hour,total_max_price,total_min_price,total_mean_price,total_price_%change,total_price_%change_trend8hr,...,sale_mean_price,sale_price_%change,sale_price_%change_trend8hr,sale_max_volume,sale_min_volume,sale_mean_volume,sale_mean_vol_%change,sale_mean_vol_%change_trend8hr,sale_vwap_price,sale_vwap_price_trend8hr
0,AUD=D5,2020,2,11,21,0.67150,0.67135,0.671448,0.000000,0.000000,...,0.671450,0.000000,0.000000,1.0,1.0,1.000000,0.000000,0.000000,11.414650,1.000000
1,AUD=D5,2020,2,11,22,0.67150,0.67140,0.671452,0.000006,0.000625,...,0.671411,-0.000058,-0.005792,2.0,1.0,1.222222,0.222222,20.000000,12.085364,1.028541
2,AUD=D5,2020,2,11,23,0.67210,0.67125,0.671769,0.000472,0.047198,...,0.671831,0.000625,0.062466,4.0,1.0,1.142857,-0.064935,-7.075472,32.920037,1.750444
3,AUD=D5,2020,2,12,0,0.67195,0.67125,0.671584,-0.000275,-0.027508,...,0.671564,-0.000396,-0.039636,3.0,1.0,1.073826,-0.060403,-6.220601,100.063092,2.557799
4,AUD=D5,2020,2,12,1,0.67335,0.67185,0.672767,0.001761,0.176016,...,0.672772,0.001798,0.179751,9.0,1.0,1.186636,0.105055,10.026631,291.990986,3.255383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8319,ZAR=D5,2020,2,28,17,15.75800,15.64400,15.710800,0.000997,0.100097,...,15.714333,0.001037,0.104120,1.0,1.0,1.000000,-0.040201,-3.965552,424.287000,0.317751
8320,ZAR=D5,2020,2,28,18,15.77000,15.73100,15.748391,0.002393,0.240165,...,15.746688,0.002059,0.206709,1.0,1.0,1.000000,0.000000,0.000000,251.947000,0.209819
8321,ZAR=D5,2020,2,28,19,15.82400,15.65400,15.761016,0.000802,0.080553,...,15.753286,0.000419,0.042102,1.0,1.0,1.000000,0.000000,0.000000,441.092000,0.370776
8322,ZAR=D5,2020,2,28,20,15.70000,15.59500,15.625591,-0.008592,-0.863609,...,15.613714,-0.008860,-0.890184,2.0,1.0,1.071429,0.071429,6.949047,218.597867,0.195564


In [569]:
agg_data_sensitive = agg_data_final[agg_data_final.Hour==21]
agg_data_nonsensitive = agg_data_final[agg_data_final.Hour!=21]

In [570]:
agg_data_sensitive.to_csv(r'C:\NICE Documents\Bank of Indonesia\agg_data_sensitive.csv', index=False)
agg_data_nonsensitive.to_csv(r'C:\NICE Documents\Bank of Indonesia\agg_data_nonsensitive.csv', index=False)

In [571]:
agg_data_final.columns

Index(['#RIC', 'Year', 'Month', 'Day', 'Hour', 'total_max_price',
       'total_min_price', 'total_mean_price', 'total_price_%change',
       'total_price_%change_trend8hr', 'total_max_volume', 'total_min_volume',
       'total_mean_volume', 'total_mean_vol_%change',
       'total_mean_vol_%change_trend8hr', 'total_vwap_price',
       'total_vwap_price_trend8hr', 'buy-sell-price_%change8hr',
       'buy-sell-volume_%change', 'buy_max_price', 'buy_min_price',
       'buy_mean_price', 'buy_price_%change', 'buy_price_%change_trend8hr',
       'buy_max_volume', 'buy_min_volume', 'buy_mean_volume',
       'buy_mean_vol_%change', 'buy_mean_vol_%change_trend8hr',
       'buy_vwap_price', 'buy_vwap_price_trend8hr', 'sale_max_price',
       'sale_min_price', 'sale_mean_price', 'sale_price_%change',
       'sale_price_%change_trend8hr', 'sale_max_volume', 'sale_min_volume',
       'sale_mean_volume', 'sale_mean_vol_%change',
       'sale_mean_vol_%change_trend8hr', 'sale_vwap_price',
       'sal